In [1]:
import pandas as pd
import numpy as np
from derive_functions.derive_date_var import derive_calender_feats
from derive_functions.data_prepare_func import create_dt,reduce_mem_usage
from derive_functions.derive_lag_mean_feats import create_lag_feats,create_lag_mean_feats
from derive_functions.derive_mean_feats_cat import get_df_cat
from derive_functions.derive_mean_feats_dept import get_df_dept
from derive_functions.derive_mean_feats_id import get_df_id
from derive_functions.derive_mean_feats_item import get_df_item
from derive_functions.derive_mean_feats_state import get_df_state
from derive_functions.derive_mean_feats_store import get_df_store
from derive_functions.derive_deviation_feats import get_deviation_feats
import gc
import datetime

# 0. 日期数据处理

In [ ]:
data_calendar = pd.read_csv('data/raw_data/calendar.csv')
data_calender_extended = pd.read_csv('data/calendar_extended.csv')
data_calendar_full = derive_calender_feats(df_calender, df_calender_extended)
data_calendar_full.to_csv('data/raw_data/calendar_full.csv',index=False)

# 1. 读取数据切分训练测试

In [3]:
data_sales_train_evaluation = pd.read_csv('data/raw_data/sales_train_evaluation.csv')

In [4]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [5]:
h = 28 
max_lags = 120
# 訓練的最後一天:d_1941
tr_last = 1941
fday = datetime.datetime(2016,5,23)

In [6]:
df_train = create_dt(is_train=True, first_day=1090, tr_last=1941, h=28)

In [7]:
df_train = reduce_mem_usage(df_train)

Mem. usage decreased to 2588.49 Mb (68.9% reduction)


In [8]:
df_test = create_dt(is_train=False, first_day= 1200, tr_last=1941, h=28)
df_test = reduce_mem_usage(df_test)

Mem. usage decreased to 719.04 Mb (69.6% reduction)


In [9]:
var_list = ['is_religious_holiday',
       'is_national_holiday', 'is_cultural_holiday', 'is_sporting_holiday',
       'num_holidays', 'ReligiousHolidayNextMonth',
       'ReligiousHolidayLastMonth', 'ReligiousHolidayThisMonth',
       'ReligiousHolidayNextWeek', 'ReligiousHolidayLastWeek',
       'ReligiousHolidayThisWeek', 'NationalHolidayNextMonth',
       'NationalHolidayLastMonth', 'NationalHolidayThisMonth',
       'NationalHolidayNextWeek', 'NationalHolidayLastWeek',
       'NationalHolidayThisWeek', 'CulturalHolidayNextMonth',
       'CulturalHolidayLastMonth', 'CulturalHolidayThisMonth',
       'CulturalHolidayNextWeek', 'CulturalHolidayLastWeek',
       'CulturalHolidayThisWeek', 'SportingHolidayNextMonth',
       'SportingHolidayLastMonth', 'SportingHolidayThisMonth',
       'SportingHolidayNextWeek', 'SportingHolidayLastWeek',
       'SportingHolidayThisWeek', 'AllHolidayNextMonth', 'AllHolidayLastMonth',
       'AllHolidayThisMonth', 'AllHolidayNextWeek', 'AllHolidayLastWeek',
       'AllHolidayThisWeek']

In [10]:
for var in var_list:
    df_train[var] = df_train[var].astype("int8")
for var in var_list:
    df_test[var] = df_test[var].astype("int8")

In [16]:
df_train.to_pickle('data/processed_data/df_train_v2.pkl')
df_test.to_pickle('data/processed_data/df_test_v2.pkl')

# 2. 变量衍生

In [3]:
data_train = pd.read_pickle('data/processed_data/df_train_v2.pkl')

In [4]:
data_train.shape

(25130703, 58)

In [3]:
# 对测试集进行衍生
# data_test = pd.read_pickle('data/processed_data/df_test_v2.pkl')
# data_train = data_test.copy(deep=True)
# del data_test
# gc.collect()

0

In [4]:
data_train['day'] = [date.day for date in data_train['date']]
data_train['week_of_yr'] = [date.week for date in data_train['date']]
data_train['is_weekend'] = np.where(data_train['wday'].isin([1,2]),1,0)
data_train['quarter'] = np.where(data_train['month'].isin([1,2,3]),1,
                                np.where(data_train['month'].isin([4,5,6]),2,
                                         np.where(data_train['month'].isin([7,8,9]),3,4)))
data_train['is_quarter_end'] = np.where((((data_train['month']==3) & (data_train['day'].isin([25,26,27,28,29,30,31])))|
                                        ((data_train['month']==6) & (data_train['day'].isin([25,26,27,28,29,30,24])))|
                                        ((data_train['month']==9) & (data_train['day'].isin([25,26,27,28,29,30,24])))|
                                        ((data_train['month']==12) & (data_train['day'].isin([25,26,27,28,29,30,31])))),1,0)
data_train['snap'] = np.where((((data_train['state_id'] == 'CA')& (data_train['snap_CA']==1))|
                              ((data_train['state_id'] == 'TX')& (data_train['snap_TX']==1))|
                              ((data_train['state_id'] == 'WI')& (data_train['snap_WI']==1))),1,0)

In [5]:
data_train.date.min()

Timestamp('2015-11-04 00:00:00')

## 2.1 mean feats

In [8]:
df_cat, df_cat_id_snap_mean, df_cat_id_weekend_mean, df_cat_id_qtr_end_mean = get_df_cat(data_train)
df_cat.to_pickle('data/processed_data/df_cat.pkl')
df_cat_id_snap_mean.to_pickle('data/processed_data/df_cat_id_snap_mean.pkl')
df_cat_id_weekend_mean.to_pickle('data/processed_data/df_cat_id_weekend_mean.pkl')
df_cat_id_qtr_end_mean.to_pickle('data/processed_data/df_cat_id_qtr_end_mean.pkl')

In [9]:
df_state, df_state_id_snap_mean, df_state_id_weekend_mean, df_state_id_qtr_end_mean = get_df_state(data_train)
df_state.to_pickle('data/processed_data/df_state.pkl')
df_state_id_snap_mean.to_pickle('data/processed_data/df_state_id_snap_mean.pkl')
df_state_id_weekend_mean.to_pickle('data/processed_data/df_state_id_weekend_mean.pkl')
df_state_id_qtr_end_mean.to_pickle('data/processed_data/df_state_id_qtr_end_mean.pkl')

In [10]:
df_store, df_store_id_snap_mean, df_store_id_weekend_mean, df_store_id_qtr_end_mean = get_df_store(data_train)
df_store.to_pickle('data/processed_data/df_store.pkl')
df_store_id_snap_mean.to_pickle('data/processed_data/df_store_id_snap_mean.pkl')
df_store_id_weekend_mean.to_pickle('data/processed_data/df_store_id_weekend_mean.pkl')
df_store_id_qtr_end_mean.to_pickle('data/processed_data/df_store_id_qtr_end_mean.pkl')

In [11]:
df_dept, df_dept_id_snap_mean, df_dept_id_weekend_mean, df_dept_id_qtr_end_mean = get_df_dept(data_train)
df_dept.to_pickle('data/processed_data/df_dept.pkl')
df_dept_id_snap_mean.to_pickle('data/processed_data/df_dept_id_snap_mean.pkl')
df_dept_id_weekend_mean.to_pickle('data/processed_data/df_dept_id_weekend_mean.pkl')
df_dept_id_qtr_end_mean.to_pickle('data/processed_data/df_dept_id_qtr_end_mean.pkl')

In [12]:
df_id, df_id_snap_mean, df_id_weekend_mean, df_id_qtr_end_mean = get_df_id(data_train)
df_id.to_pickle('data/processed_data/df_id.pkl')
df_id_snap_mean.to_pickle('data/processed_data/df_id_snap_mean.pkl')
df_id_weekend_mean.to_pickle('data/processed_data/df_id_weekend_mean.pkl')
df_id_qtr_end_mean.to_pickle('data/processed_data/df_id_qtr_end_mean.pkl')

In [13]:
df_item, df_item_snap_mean, df_item_weekend_mean, df_item_qtr_end_mean,df_item_price_mode = get_df_item(data_train)
df_item.to_pickle('data/processed_data/df_item.pkl')
df_item_snap_mean.to_pickle('data/processed_data/df_item_snap_mean.pkl')
df_item_weekend_mean.to_pickle('data/processed_data/df_item_weekend_mean.pkl')
df_item_qtr_end_mean.to_pickle('data/processed_data/df_item_qtr_end_mean.pkl')
df_item_price_mode.to_pickle('data/processed_data/df_item_price_mode.pkl')

## 2.2 lag feats

In [6]:
data_train = create_lag_feats(data_train,recursive=False)

## 2.3 lag mean feats

In [7]:
data_train = create_lag_mean_feats(data_train,recursive=False)

get_mean_feats...
generating sales_per_id_lag_7_r_7_mean
generating sales_per_id_lag_7_r_28_mean
generating sales_per_id_lag_7_r_90_mean
generating sales_per_id_lag_28_r_7_mean
generating sales_per_id_lag_28_r_28_mean
generating sales_per_id_lag_28_r_90_mean
generating revenue_per_id_lag_7_r_7_mean
generating revenue_per_id_lag_7_r_28_mean
generating revenue_per_id_lag_7_r_90_mean
generating revenue_per_id_lag_28_r_7_mean
generating revenue_per_id_lag_28_r_28_mean
generating revenue_per_id_lag_28_r_90_mean
get_median_feats...
generating sell_price_per_id_lag_7_r_7_median
generating sell_price_per_id_lag_7_r_28_median
generating sell_price_per_id_lag_7_r_90_median
generating sell_price_per_id_lag_28_r_7_median
generating sell_price_per_id_lag_28_r_28_median
generating sell_price_per_id_lag_28_r_90_median
get deviation feats...


In [8]:
data_train = reduce_mem_usage(data_train, verbose=True)

Mem. usage decreased to 1025.26 Mb (14.4% reduction)


In [9]:
# data_train.to_pickle('data/processed_data/df_train_before_merge_v2.pkl')
data_train.to_pickle('data/processed_data/df_test_before_merge_v2.pkl')

## 2.4 merge 1,2,3

In [3]:
data_train = pd.read_pickle('data/processed_data/df_train_before_merge_v2.pkl')

In [5]:
# data_train = pd.read_pickle('data/processed_data/df_train_before_merge.pkl')

df_cat = pd.read_pickle('data/processed_data/mean_feats/df_cat.pkl')
df_cat_id_snap_mean = pd.read_pickle('data/processed_data/mean_feats/df_cat_id_snap_mean.pkl')
df_cat_id_weekend_mean = pd.read_pickle('data/processed_data/mean_feats/df_cat_id_weekend_mean.pkl')
df_cat_id_qtr_end_mean = pd.read_pickle('data/processed_data/mean_feats/df_cat_id_qtr_end_mean.pkl')

df_state = pd.read_pickle('data/processed_data/mean_feats/df_state.pkl')
df_state_id_snap_mean = pd.read_pickle('data/processed_data/mean_feats/df_state_id_snap_mean.pkl')
df_state_id_weekend_mean = pd.read_pickle('data/processed_data/mean_feats/df_state_id_weekend_mean.pkl')
df_state_id_qtr_end_mean = pd.read_pickle('data/processed_data/mean_feats/df_state_id_qtr_end_mean.pkl')


df_store = pd.read_pickle('data/processed_data/mean_feats/df_store.pkl')
df_store_id_snap_mean = pd.read_pickle('data/processed_data/mean_feats/df_store_id_snap_mean.pkl')
df_store_id_weekend_mean = pd.read_pickle('data/processed_data/mean_feats/df_store_id_weekend_mean.pkl')
df_store_id_qtr_end_mean = pd.read_pickle('data/processed_data/mean_feats/df_store_id_qtr_end_mean.pkl')

df_dept = pd.read_pickle('data/processed_data/mean_feats/df_dept.pkl')
df_dept_id_snap_mean = pd.read_pickle('data/processed_data/mean_feats/df_dept_id_snap_mean.pkl')
df_dept_id_weekend_mean = pd.read_pickle('data/processed_data/mean_feats/df_dept_id_weekend_mean.pkl')
df_dept_id_qtr_end_mean = pd.read_pickle('data/processed_data/mean_feats/df_dept_id_qtr_end_mean.pkl')

df_id = pd.read_pickle('data/processed_data/mean_feats/df_id.pkl')
df_id_snap_mean = pd.read_pickle('data/processed_data/mean_feats/df_id_snap_mean.pkl')
df_id_weekend_mean = pd.read_pickle('data/processed_data/mean_feats/df_id_weekend_mean.pkl')
df_id_qtr_end_mean = pd.read_pickle('data/processed_data/mean_feats/df_id_qtr_end_mean.pkl')

df_item = pd.read_pickle('data/processed_data/mean_feats/df_item.pkl')
df_item_id_snap_mean = pd.read_pickle('data/processed_data/mean_feats/df_item_snap_mean.pkl')
df_item_id_weekend_mean = pd.read_pickle('data/processed_data/mean_feats/df_item_weekend_mean.pkl')
df_item_id_qtr_end_mean = pd.read_pickle('data/processed_data/mean_feats/df_item_qtr_end_mean.pkl')
df_item_id_price_mode = pd.read_pickle('data/processed_data/mean_feats/df_item_price_mode.pkl')
df_item_id_price_mode = reduce_mem_usage(df_item_id_price_mode, verbose=True)


df_cat = reduce_mem_usage(df_cat, verbose=True)
df_state = reduce_mem_usage(df_state, verbose=True)
df_store = reduce_mem_usage(df_store, verbose=True)
df_dept = reduce_mem_usage(df_dept, verbose=True)
df_id = reduce_mem_usage(df_id, verbose=True)
df_item = reduce_mem_usage(df_item, verbose=True)


df_cat_id_snap_mean = reduce_mem_usage(df_cat_id_snap_mean, verbose=True)
df_state_id_snap_mean = reduce_mem_usage(df_state_id_snap_mean, verbose=True)
df_store_id_snap_mean = reduce_mem_usage(df_store_id_snap_mean, verbose=True)
df_dept_id_snap_mean = reduce_mem_usage(df_dept_id_snap_mean, verbose=True)
df_id_snap_mean = reduce_mem_usage(df_id_snap_mean, verbose=True)
df_item_id_snap_mean = reduce_mem_usage(df_item_id_snap_mean, verbose=True)

df_item_id_weekend_mean = reduce_mem_usage(df_item_id_weekend_mean, verbose=True)
df_id_weekend_mean = reduce_mem_usage(df_id_weekend_mean, verbose=True)
df_dept_id_weekend_mean = reduce_mem_usage(df_dept_id_weekend_mean, verbose=True)
df_store_id_weekend_mean = reduce_mem_usage(df_store_id_weekend_mean, verbose=True)
df_state_id_weekend_mean = reduce_mem_usage(df_state_id_weekend_mean, verbose=True)
df_cat_id_weekend_mean = reduce_mem_usage(df_cat_id_weekend_mean, verbose=True)

df_item_id_qtr_end_mean = reduce_mem_usage(df_item_id_qtr_end_mean, verbose=True)
df_id_qtr_end_mean = reduce_mem_usage(df_id_qtr_end_mean, verbose=True)
df_dept_id_qtr_end_mean = reduce_mem_usage(df_dept_id_qtr_end_mean, verbose=True)
df_store_id_qtr_end_mean = reduce_mem_usage(df_store_id_qtr_end_mean, verbose=True)
df_state_id_qtr_end_mean = reduce_mem_usage(df_state_id_qtr_end_mean, verbose=True)
df_cat_id_qtr_end_mean = reduce_mem_usage(df_cat_id_qtr_end_mean, verbose=True)

Mem. usage decreased to 34.54 Mb (27.3% reduction)
Mem. usage decreased to  0.00 Mb (35.7% reduction)
Mem. usage decreased to  0.00 Mb (46.5% reduction)
Mem. usage decreased to  0.00 Mb (46.5% reduction)
Mem. usage decreased to  0.00 Mb (46.5% reduction)
Mem. usage decreased to  3.84 Mb (46.8% reduction)
Mem. usage decreased to  0.38 Mb (46.3% reduction)
Mem. usage decreased to  0.00 Mb (21.3% reduction)
Mem. usage decreased to  0.00 Mb (26.0% reduction)
Mem. usage decreased to  0.00 Mb (40.1% reduction)
Mem. usage decreased to  0.00 Mb (36.5% reduction)
Mem. usage decreased to  1.22 Mb (34.4% reduction)
Mem. usage decreased to  0.06 Mb (50.0% reduction)
Mem. usage decreased to  0.06 Mb (50.0% reduction)
Mem. usage decreased to  1.22 Mb (34.4% reduction)
Mem. usage decreased to  0.00 Mb (36.5% reduction)
Mem. usage decreased to  0.00 Mb (40.1% reduction)
Mem. usage decreased to  0.00 Mb (26.0% reduction)
Mem. usage decreased to  0.00 Mb (21.3% reduction)
Mem. usage decreased to  0.06 M

### 2.4.1 删除多余的列，不然内存报错

In [11]:
for var in df_cat.columns:
    if ('avg_revenue_per_cat_' in var) or ('qtr' in var):
        del df_cat[var]
        gc.collect()
           
for var in df_dept.columns:
    if ('avg_revenue_per_dept_' in var) or ('qtr' in var):
        del df_dept[var]
        gc.collect()
        
for var in df_store.columns:
    if ('avg_revenue_per_store_' in var) or ('qtr' in var):
        del df_store[var]
        gc.collect()
        
        
for var in df_state.columns:
    if ('avg_revenue_per_state_' in var) or ('qtr' in var):
        del df_state[var]
        gc.collect()
        
for var in df_item.columns:
    if ('avg_revenue_per_item_' in var) or ('qtr' in var):
        del df_item[var]
        gc.collect()
        
for var in df_id.columns:
    if ('avg_revenue_per_id_' in var) or ('qtr' in var):
        del df_id[var]
        gc.collect()

### 2.4.2 merge p1

In [12]:
df_train = pd.merge(data_train, df_store, on='store_id', how='left')
del data_train
gc.collect()
df_train = pd.merge(df_train, df_dept, on='dept_id', how='left')
df_train = pd.merge(df_train, df_id, on='id', how='left')

### 2.4.3 merge p2

In [13]:
df_train = pd.merge(df_train, df_store_id_snap_mean, on=['snap','store_id'], how='left')
df_train = pd.merge(df_train, df_store_id_weekend_mean, on=['is_weekend','store_id'], how='left')
df_train = pd.merge(df_train, df_store_id_qtr_end_mean, on=['is_quarter_end','store_id'], how='left')
df_train = pd.merge(df_train, df_dept_id_snap_mean, on=['snap','dept_id'], how='left')
df_train = pd.merge(df_train, df_dept_id_weekend_mean, on=['is_weekend','dept_id'], how='left')
df_train = pd.merge(df_train, df_dept_id_qtr_end_mean, on=['is_quarter_end','dept_id'], how='left')
df_train = pd.merge(df_train, df_id_snap_mean, on=['snap','id'], how='left')
df_train = pd.merge(df_train, df_id_weekend_mean, on=['is_weekend','id'], how='left')
df_train = pd.merge(df_train, df_id_qtr_end_mean, on=['is_quarter_end','id'], how='left')

## 2.5. deviation feats

In [15]:
df_train = get_deviation_feats(df_train)
df_train = reduce_mem_usage(df_train)

Mem. usage decreased to 2582.96 Mb (0.0% reduction)


In [16]:
# df_train.to_pickle('data/processed_data/df_train_var_all_0625.pkl')
df_train.to_pickle('data/processed_data/df_test_var_all_0625.pkl')

## 2.6 合并每个item的价格众数

In [3]:
df_train = pd.read_pickle('data/processed_data/df_train_var_all_0625.pkl')
df_train = df_train[df_train.date>='2014-05-20']
gc.collect()
useless_cols = ['d','wm_yr_wk','weekday']
for var in useless_cols:
    del df_train[var]
    gc.collect()

In [ ]:
df_item_id_price_mode = pd.read_pickle('data/processed_data/mean_feats/df_item_price_mode_train.pkl')
df_item_id_price_mode = reduce_mem_usage(df_item_id_price_mode, verbose=True)
df_train = pd.merge(df_train,df_item_id_price_mode[['item_id', 'date', 'item_sell_price_mode']],
                    on=['item_id','date'],how='left')
gc.collect()

df_train['item_sell_price_mode_dev'] = (df_train['sell_price']-df_train['item_sell_price_mode'])/df_train['item_sell_price_mode']

In [ ]:
# 输出训练数据集
df_train.to_pickle('data/processed_data/df_train_var_all_0626.pkl')
df_train[df_train.state_id=='TX'].to_pickle('data/processed_data/df_train_var_all_0626_TX_subset.pkl')
df_train[df_train.state_id=='WI'].to_pickle('data/processed_data/df_train_var_all_0626_WI_subset.pkl')
df_train[(df_train.state_id=='CA') & (df_train.date>='2014-09-06')].to_pickle('data/processed_data/df_train_var_all_0626_CA_subset.pkl')

In [5]:
# 输出测试数据集
df_test = pd.read_pickle('data/processed_data/df_test_var_all_0625.pkl')
df_item_price_mode = df_test[['item_id', 'date', 'sell_price']].groupby(['item_id', 'date'], as_index=False).agg(
    lambda x: x.value_counts().index[0])
df_item_price_mode.rename(columns={'sell_price': 'item_sell_price_mode'}, inplace=True)
df_item_price_mode.to_pickle('data/processed_data/mean_feats/df_item_price_mode_test.pkl')

df_test = pd.merge(df_test,df_item_price_mode[['item_id', 'date', 'item_sell_price_mode']],
                    on=['item_id','date'],how='left')
df_test['item_sell_price_mode_dev'] = (df_test['sell_price']-df_test['item_sell_price_mode'])/df_test['item_sell_price_mode']
df_test.to_pickle('data/processed_data/df_test_var_all_0626.pkl')